# 3.6 EXERCISES

## 3.1

** Can all matrices $A \in \mathbb{R}^{n \times n}$ be factored $A = LU$?  Why or why not?**

No. Normal LU decompostion cannot deal with some matrices (like the one in section 3.2), because in which case a diagonal element becomes 0 while forward-substitution.  However, using permutation matrix $P$ allows factorization to $A = PLU$. 

## 3.2

**Solve the following system of equations using Gaussian elimination, writing the corresponding elimination matrix of each step:$$
\begin{pmatrix}
  2 & 4 \\
  3 & 5 \\
\end{pmatrix}
\begin{pmatrix}
  x \\
  y \\
\end{pmatrix}
 = \begin{pmatrix}
 2 \\
 4 \\
\end{pmatrix}
$$Factor the matrix on the left-hand side as a product $A = LU$.**

Using augmented matrix,

\begin{equation}
\begin{split}
  \left(\begin{array}{cc|c}
    2 & 4 & 2 \\
    3 & 5 & 4 \\
  \end{array}\right)
  &\rightarrow
  \left(\begin{array}{cc|c}
    2 & 4 & 2 \\
    0 & -1 & 1 \\
  \end{array}\right) \qquad
  \left[\left(\begin{array}{cc}
    1 & 0 \\
    -3/2 & 1 \\
  \end{array}\right)\right] \\
  &\rightarrow
  \left(\begin{array}{cc|c}
    1 & 2 &  1 \\
    0 & 1 & -1 \\
  \end{array}\right) \qquad
  \left[\left(\begin{array}{cc}
    1/2 & 0 \\
    0 & -1 \\
  \end{array}\right)\right] \\
  &\rightarrow
  \left(\begin{array}{cc|c}
    1 & 0 & 3 \\
    0 & 1 & -1 \\
  \end{array}\right) \qquad
  \left[\left(\begin{array}{cc}
    1 & -2 \\
    0 &  1 \\
  \end{array}\right)\right] \\
\end{split}
\end{equation}

This process gives us the hint of LU decomposition:

\begin{equation}
\begin{split}
  &\left(\begin{array}{cc}
    1 & 0 \\
    -3/2 & 1 \\
  \end{array}\right)
  \left(\begin{array}{cc}
    2 & 4 \\
    3 & 5 \\
  \end{array}\right)
  =
  \left(\begin{array}{cc}
    2 & 4 \\
    0 & -1 \\
  \end{array}\right) \\
  &\iff
  \left(\begin{array}{cc}
    2 & 4 \\
    3 & 5 \\
  \end{array}\right)
  =
  \left(\begin{array}{cc}
    1 & 0 \\
    3/2 & 1 \\
  \end{array}\right)
  \left(\begin{array}{cc}
    2 & 4 \\
    0 & -1 \\
  \end{array}\right) \\
\end{split}
\end{equation}

## 3.3

**Factor the following matrix $A$ as $A = LU$.$$
\begin{pmatrix}
  1 & 2 & 7 \\
  3 & 5 & -1 \\
  6 & 1 & 4 \\
\end{pmatrix}
$$**

In [15]:
import numpy as np

def lu_without_pivoting(A, compact=False):
    """Factors A to A = LU"""
    assert A.ndim == 2
    A = A.astype(float)
    n1, n2 = A.shape
    for p in range(n1):
        for r in range(p+1, n1):
            s = -A[r, p] / A[p, p]
            A[r, p] = -s
            A[r, p+1:] += s * A[p, p+1:]
    if compact:
        return A
    else:
        n_min = min(n1, n2)
        L = np.fromfunction(lambda i, j : A*(i > j) + np.eye(n1, n_min),
                            shape=(n1, n_min), dtype=float)
        U = np.fromfunction(lambda i, j : A*(i <= j),
                            shape=(n_min, n2), dtype=float)
    return L, U

In [17]:
A = np.array([
    [1, 2, 7],
    [3, 5, -1],
    [6, 1, 4]
])
L, U = lu_without_pivoting(A)
print("L =\n", L)
print("U =\n", U)

L =
 [[  1.   0.   0.]
 [  3.   1.   0.]
 [  6.  11.   1.]]
U =
 [[   1.    2.    7.]
 [   0.   -1.  -22.]
 [   0.    0.  204.]]


## 3.4

**Modify the code in Figure 3.1 to include partial pivoting.**

In [11]:
import numpy as np

def forward_substitution_pivoting(A, b):
    """ Cnverts a system Ax = b to an upper-triangular system Ux = y.
    
    Positional arguments:
        A -- Invertible numpy array. A.shape == (n, n)
        b -- 1-dimensional Numpy array. b.shape == (n, )
    """
    assert A.ndim == 2 and b.ndim == 1 and A.shape[0] == A.shape[1] == b.size
    
    n = b.size
    
    # U will be upper triangular at completion.
    U = A.astype(float)
    y = b.astype(float)
    
    # Iterate over current pivot row p.
    for p in range(n):
        pivot = np.argmax(np.absolute(U[p:, p])) + p
        if pivot != p:
            U[p], U[pivot] = U[pivot].copy(), U[p].copy()
            y[p], y[pivot] = y[pivot], y[p]
        s = U[p, p]
        if s != 0:
            y[p] /= s
            U[p] /= s

            for r in range(p + 1, n):
                s = U[r, p]
                y[r] -= s * y[p]
                U[r] -= s * U[p]
    return U, y

def backward_substitution(U, y):
    """Solves upper-triangulate systems Ux = y for x"""
    assert U.ndim == 2 and y.ndim == 1 and U.shape[0] == U.shape[1] == y.size
    
    n = y.size
    x = y.copy()
    for p in range(n-1, -1, -1):
        x[p] -= U[p, p+1:] @ x[p+1:]
    return x

In [12]:
A = np.array([
    [0, 1, -1],
    [3, -1, 1],
    [1, 1, -2]
])
b = np.array([-1, 4, -3])
backward_substitution(*forward_substitution_pivoting(A, b))

array([ 1.,  2.,  3.])

## 3.5

**The discussion in §3.4.3 includes an example of a 2 × 2 matrix $A$ for which Gaussian elimination without pivoting fails.  In this case, the issue was resolved by introducing partial pivoting.  If exact arithmetic is implemented to alleviate rounding error, does there exist a matrix for which Gaussian elimination fails unless _full_ rather than partial pivoting is implemented? Why or why not?**

Yes, Gaussian elimination fails when the matrix is not invertible, such as$$
  \begin{pmatrix}
    1 & 1 & 1 & 1  \\
    0 & 0 & 1 & 1  \\
    0 & 0 & 1 & 1  \\
    0 & 0 & 0 & 1  \\
  \end{pmatrix}.
$$
If the matrix is invertible, there must exist a unique solution so that the Gaussian elimination will succeed. 

## 3.6

**Numerical algorithms appear in many components of simulation software for quantum physics.  The Schrodinger equation and others involve complex numbers in $\mathbb{C}$, however, so we must extend the machinery we have developed for solving linear systems of equations to this case.  Recall that a complex number $x \in \mathbb{C}$ can be written as $x = a + bi$, where $a, b \in \mathbb{R}$ and $i = \sqrt{-1}$.  Suppose we wish to solve $A\vec{x} = \vec{b}$, but now $A \in \mathbb{C}^{n \times n}$ and $x, \vec{b} \in \mathbb{C}^n$.  Explain how a linear solver that takes only _real-valued_ system can be used to solve this equation.**

$A$ and $\vec{x}, \vec{b}$ can be decomposed into:

\begin{equation}
\begin{split}
  A &= A_1 + A_2i, &\qquad A_1,A_2 \in \mathbb{R}^{n\times n} \\
  \vec{x} &= \vec{x}_1 + \vec{x}_2i, &\qquad \vec{x}_1, \vec{x}_2 \in \mathbb{R}^n \\
  \vec{b} &= \vec{b}_1 + \vec{b}_2i, &\qquad \vec{b}_1, \vec{b}_2 \in \mathbb{R}^n \\
\end{split}
\end{equation}

Then $A\vec{x} = \vec{b}$ can be written as:

\begin{equation}
  (A_1 + A_2i)(\vec{x}_1 + \vec{x}_2i) = A_1\vec{x}_1 - A_2\vec{x}_2 + (A_2\vec{x}_1 + A_1\vec{x}_2)i = \vec{b}_1 + \vec{b}_2i \\
\end{equation}

Hence, the linear system we solve is:

\begin{equation}
\begin{split}
  A_1\vec{x}_1 - A_2\vec{x}_2 &= \vec{b}_1 \\
  A_2\vec{x}_1 + A_1\vec{x}_2 &= \vec{b}_2 \\
\end{split}
\implies
\begin{pmatrix}
  A_1 & -A_2 \\
  A_2 &  A_1 \\
\end{pmatrix}
\begin{pmatrix}
  \vec{x}_1 \\
  \vec{x}_2 \\
\end{pmatrix}
=
\begin{pmatrix}
  \vec{b}_1 \\
  \vec{b}_2 \\
\end{pmatrix}
\end{equation}

This is a $2n\times2n$ real valued system.

## 3.7

**Suppose $A \in \mathbb{R}^{n\times n}$ is invertible.  Show that $A^{-1}$ can be obtained using Gaussian elimination on augumented matrix**

\begin{equation}
\left(\begin{array}{c|c}
  A & I_{n\times n} \\
\end{array}\right).
\end{equation}

The definition of invertible matrix $A^{-1}$ is

\begin{equation}
  AA^{-1} = I_{n\times n}
\end{equation}

By writing $A^{-1} = (\vec{x_1}, \vec{x_2}, \dots, \vec{x_n})$ and $I_{n\times n} = (\vec{e_1}, \vec{e_2}, \dots \vec{e_n})$, we can decompose this linear system to

\begin{equation}
  A\vec{x}_k = \vec{e}_k, \qquad (k = 1, 2, \dots, n)
\end{equation}

Using augumentation matrix, this is written like

\begin{equation}
\left(\begin{array}{c|cccc}
  A & \vec{e}_1 & \vec{e}_2 & \cdots & \vec{e}_n \\
\end{array}\right)
  =
\left(\begin{array}{c|c}
  A & I_{n\times n} \\
\end{array}\right).
\end{equation}

This system can be solved using Gaussian elimination.

## 3.8

**Show that if $L$ is an invertible lower-triangular matrix, none of its diagonal elements can be zero.  How does this lemma affect the construction in §3.5.3?**

The determinant of a lower-tirangular matrix $L$ is the multiplication of its diagonal element . (This is apparent because row or column elimination on $L$ will yield diagonal matrix without changing the determinant.) If an lower-triangular matrix $L$ includes zero in its diagonal element, the determinant is zero and it is not invertible, which doesn't meet the provided condition.

## 3.9

**Show that the inverse of an (invertible) lower-triangular matrix is lower triangular.**

From problem 3.8, we know that all of the diagonal element of an invertible lower-triangular matrix $L$ are not zero. Therefore, we only need scale matrix and elimination matrix to make $L$ identity.  If we denote scale or elimination matrix as $M_i$, we can write:

\begin{equation}
\begin{split}
  &M_n M_{n-1} \cdots M_2 M_1 A = I_{n\times n} \\
  &\implies A^{-1} = M_n M_{n-1} \cdots M_2 M_1
\end{split}
\end{equation}

All $M_i$ are lower-triangular matrices.
Hence, according to the Proposition 3.1 in pg.60, $A^{-1}$ is also lower-triangular.

##  3.10

**Show that any invertible matrix $A \in \mathbb{R}^{n \times n}$ with $a_{11} = 0$ cannot have a factorization $A = LU$ for lower-triangular $L$ and upper-triangular $U$.**

An invertible $A$ with $a_{11} = 0$ cannot be factorized to $A = LU$ because forward-substitution fails at the very first step.  However, as I said in Problem 3.1, permutation matrices $P$ will allow the factorization $A = PLU$.

## 3.11

**Show how the LU factorization of $A\in\mathbb{R}^{n \times n}$ can be used to compute the determinant of $A$**

Since row or column elimnation on any matrix won't change its determinant, the determinant of a lower-or-upper-triangular matrix is the multiplication of all of the diagonal elemnts.  Hence, the determinant of $A = LU$ is the product of each diagonal element of $L$ and $U$. 